In [1]:
# -*- coding:utf-8 -*-
# 数据加载、存储与文件格式
# read_csv: 从文本、URL、文本型对象中加载带分隔符的数据，默认为逗号
# read_table: 从文本、URL、文本型对象中加载带分隔符的数据，默认为制表符（\t）
# read_fwf: 读取定宽列个事数据（没有分隔符）
# read_clipboard: 读取剪切板中的数据，可以看作read_table的剪贴板版，再将网页转化为表格很有用
import pandas as pd
from pandas import Series, DataFrame

In [3]:
# 解析器函数参数： header = None, sep = 字符或者正则表达式 , names = 列名, index_col = 选做索引的列
# skiprows = 跳过行的list -- nrows = 需要读取的列数， comment : 用于将注释信息从行尾拆分出去的字符
# 解析日期： parse_dates, keep_data_col, date_parser dayfirst : 有歧义日期按照日月年解析
# converters : 将列号或者行号与函数之间映射形成字典  encoding
# iterator 返回TextParser逐块读取文件，chunksize 文件块的大小

In [4]:
## 1.读写文本格式数据
df = pd.read_csv('D:/Workshop/PyFile/pydata-book-master/ch06/ex1.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [2]:
address = r'D:/Workshop/PyFile/pydata-book-master/ch06/'

In [6]:
pd.read_table(address + 'ex1.csv', sep = ',')  # 用read_table默认是制表符，所以这里要特意用sep设定','

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [7]:
pd.read_csv(address + 'ex2.csv', header = None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [8]:
pd.read_csv(address + 'ex2.csv', names = ['a', 'b', 'c', 'd', 'message'])  # 默认索引

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [9]:
names = ['a', 'b', 'c', 'd', 'message']
pd.read_csv(address + 'ex2.csv', names = names, index_col = 'message')  # 指定message作为索引

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [10]:
parsed = pd.read_csv(address + 'csv_mindex.csv', index_col = ['key1', 'key2'])  # 指定其中两列做多层次索引
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In [11]:
result = pd.read_csv(address + 'ex3.txt', sep = '\s+')  
# 由于列名比列的数目少一，所以read_table推断第一列为DataFrame的索引
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


In [12]:
pd.read_csv(address + 'ex4.csv', skiprows = [0, 2, 3])  # 跳过几行不读取

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [13]:
result = pd.read_csv(address + 'ex5.csv')  # 存在缺失值，表示可以为NA,-1.#IND, NULL
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [14]:
pd.isnull(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


In [15]:
result = pd.read_csv(address + 'ex5.csv', na_values = ['NULL'])  # 表示用于接收缺失值的字符串
pd.isnull(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


In [16]:
# 可以用一个字典为各列指定不同的NA标记值
sentinels = {'message':['foo','NA'], 'something':['two']}
pd.read_csv(address + 'ex5.csv', na_values = sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


In [17]:
result = pd.read_csv(address + 'ex6.csv')  # 大文件
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
5,1.817480,0.742273,0.419395,-2.251035,Q
6,-0.776764,0.935518,-0.332872,-1.875641,U
7,-0.913135,1.530624,-0.572657,0.477252,K
8,0.358480,-0.497572,-0.367016,0.507702,S
9,-1.740877,-1.160417,-1.637830,2.172201,G


In [18]:
# 只读取几个
pd.read_csv(address + 'ex6.csv', nrows = 5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


In [19]:
chunker = pd.read_csv(address + 'ex6.csv', chunksize = 1000)  # 逐块读取
chunker  # TextParser对象

In [20]:
chunker = pd.read_csv(address + 'ex6.csv', chunksize = 1000)
tot = Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value = 0)
tot = tot.sort_values(ascending = False)
tot[:10]

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

In [21]:
## 2. 数据写入文本格式
data = pd.read_csv(address + 'ex5.csv')
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [22]:
data.to_csv(address + 'out.csv')

In [24]:
data.to_csv(sys.stdout, sep = '|')

NameError: name 'sys' is not defined

In [3]:
# 手工处理分隔符格式
# 这个例子是有畸形行
import csv
f = open(address + 'ex7.csv')  # 逐行读取，每行返回一个元组
reader = csv.reader(f)  # reader这里是个迭代对象

NameError: name 'address' is not defined

In [5]:
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3', '4']


In [7]:
lines = list(csv.reader(open(address + 'ex7.csv')))
header, values = lines[0], lines[1:]
data_dict = {h: v for h, v in zip(header, zip(*values))}  # 这是一个字典推导式
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

In [6]:
# 如何处理分隔符格式比较奇葩的数据
class my_dialect(csv.Dialect):
    lineterminator = '\n'
    delimiter = ';'
    quotechar = ''
reader = csv.reader(f, dialect = my_dialect)

TypeError: argument 1 must be an iterator

In [5]:
# CSV语支选项
# delimiter  用于分个字段的单字符字符串
# lineterminator  用于写操作的行结束符， 默认'\r\n', 读操作忽略此选项，能识别跨平台的行结束符
# quotechar  用于带有特殊字符（如分隔符）的字段的引用符号
# quoting  引用约定
# skipinitialspace  忽略分隔符后面的空白符， 默认F
# doublequote  如何处理字段内的引用符号， T：双写 
# escapechar 用于对分隔符进行转义的字符串

# 要手工输出分隔符文件， 可以使用csv.writer -- 接受一个已打开且可写的文件对象以及跟csv.reader相同的那些语支和格式化选项：
with open('mydata.csv', 'w') as f:
    writer = csv.writer(f, dialect = my_dialect)
    writer.writerow(('one','two','three'))
    writer.writerow(('1','2','3'))
    writer.writerow(('4','5','6'))
    writer.writerow(('7','8','9'))


NameError: name 'my_dialect' is not defined

In [13]:
# JSON
# JSON --基本类型有对象（字典），数组（列表），字符串，数值，布尔值以及null，对象中所有的键必须是字符串。开头结尾都是'''    '''
# json.loads  --将JSON字符串转化成Python，  json.dumps -- 将python对象转化成JSON
obj = """
{"name":"Wes",
"place_lived":["United States", "Spain","Germany"],
"pet":null,
"siblings":[{"name":"Scott","age":25,"pet":"Zuko"},
{"name":"Katie","age":33,"pet":"Cisco"}]
}
"""

In [14]:
import json
result = json.loads(obj)
result

{'name': 'Wes',
 'pet': None,
 'place_lived': ['United States', 'Spain', 'Germany'],
 'siblings': [{'age': 25, 'name': 'Scott', 'pet': 'Zuko'},
  {'age': 33, 'name': 'Katie', 'pet': 'Cisco'}]}

In [15]:
asjson = json.dumps(result)
asjson

'{"place_lived": ["United States", "Spain", "Germany"], "siblings": [{"pet": "Zuko", "name": "Scott", "age": 25}, {"pet": "Cisco", "name": "Katie", "age": 33}], "pet": null, "name": "Wes"}'

In [16]:
# 向DataFrame构造器传入一组JSON对象，并选取数据字段的子集
siblings = DataFrame(result['siblings'], columns = ['name', 'age'])
siblings

,name,age
0,Scott,25
1,Katie,33


In [4]:
# XML与HTML
from lxml.html import parse  # LXML库
from urllib.request import urlopen

In [5]:
parsed = parse(urlopen('http://finance.yahoo.com/q/op?s=AAPL+Options'))
doc = parsed.getroot()
links = doc.findall('.//a')
links[15:20]  # 展现HTML元素的对象

[<Element a at 0x1bee5ffeb88>,
 <Element a at 0x1bee5ffebd8>,
 <Element a at 0x1bee5ffec28>,
 <Element a at 0x1bee5ffec78>,
 <Element a at 0x1bee5ffecc8>]

In [6]:
# 得到URL和链接文本-- get:(for URL)  text_content(for text)
lnk = links[28]
lnk

<Element a at 0x1bee5ffef98>

In [7]:
lnk.get('href')

'/quote/ES=F?p=ES=F'

In [8]:
lnk.text_content()

'S&P Futures'

In [9]:
# 通过list comprehension获取文档中全部URL
urls = [lnk.get('href') for lnk in doc.findall('.//a')]
urls[-10:]

['https://smallbusiness.yahoo.com',
 'https://help.yahoo.com/kb/index?page=content&y=PROD_FIN_DESK&locale=en_US&id=SLN2310',
 'https://help.yahoo.com/kb/index?page=content&y=PROD_FIN_DESK&locale=en_US',
 'https://yahoo.uservoice.com/forums/382977',
 'http://info.yahoo.com/privacy/us/yahoo/',
 'http://info.yahoo.com/relevantads/',
 'http://info.yahoo.com/legal/us/yahoo/utos/utos-173.html',
 'https://twitter.com/YahooFinance',
 'https://facebook.com/yahoofinance',
 'http://yahoofinance.tumblr.com']

In [11]:
from pandas.io.parsers import TextParser  # 自动类型转换

In [19]:
# XML --常见支持分层、嵌套数据以及元数据的结构化数据格式。
# 使用lxml.objectify来操控XML数据的接口
from lxml import objectify

In [20]:
# 二进制数据格式
# 实现数据的二进制格式存储 -- 使用pickle序列化--to_pickle方法
frame = pd.read_csv(address + 'ex1.csv')
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [22]:
frame.to_pickle(address + 'frame_pickle')  # 存储成二进制格式

In [25]:
# 导回Python格式
pd.load(address + 'frame_pickle')

AttributeError: module 'pandas' has no attribute 'dump'

In [6]:
xls_file = pd.ExcelFile(address + 'ex1.xlsx')  # 获取EXCEL  --xlsx
table = xls_file.parse('Sheet1')   # 获取xlsx里的sheet
table

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [8]:
# HTML 和 web API
import requests
url = 'https://twitter.com/search?q=python%20pandas&src=typd'
resp = requests.get(url)
resp

<Response [200]>

In [10]:
import json
data = json.loads(resp.text)
data.keys()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [11]:
# 数据库SQL
import sqlite3
query = '''
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
c REAL, d INTEGER
);'''
con = sqlite3.connect(':memory:')
con.execute(query)
con.commit()

In [12]:
data = [('Atlanta','Georgia',1.25,6),
       ('Tallahassee','Florida',2.6,3),
       ('Sacramento','California',1.7,5)]
stmt = 'INSERT INTO test VALUES(?,?,?,?)'
con.executemany(stmt, data)
con.commit()

In [14]:
cursor = con.execute('select * from test')
rows = cursor.fetchall()
rows  # 大部分SQL驱动器都返回一个元祖列表

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

In [15]:
cursor.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [18]:
# 生成DataFrame
DataFrame(rows, columns = zip(*cursor.description)[0])

TypeError: 'zip' object is not subscriptable

In [21]:
# 简化过程
import pandas.io.sql as sql
sql.read_sql_query('select * from test', con)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5
